In [1]:
import pandas as pd
from collections import defaultdict
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## This is for certification data analysis of the following states: New Mexico, North Carolina, Minnesota, Indiana, Kansas, Mississippi and Illinois

### The questions to answer:
#### 1. How far back does the data go?
#### 2. How many currently-active officers have had 3+ employers? 5+? More?
#### 3. How many currently-active officers were previously listed as having been fired or terminated?
#### 4. For officers who were previously terminated, and/or officers who have 3/5+ former employers, are there agencies where they seem to end up more often than not?

***

#### **Some functions that are used repeatedly**

In [2]:
# getting number of departments each officer has been a part of and creating a new column that lists that count
# INPUTS
# df: the state's cert data dataframe
# id_column: the name of the column in this dataframe that refers to an officer's unique id
# agency_column: the name of the column that refers to the department name
def get_dept_counts(df, id_column, agency_column):
    officer_dept_counts = df.value_counts(id_column)
    dept_counts_dict = pd.DataFrame({id_column:officer_dept_counts.index, 'number_of_depts':officer_dept_counts.values}).set_index(id_column)['number_of_depts'].to_dict()
    df['number_of_depts'] = df[id_column].map(dept_counts_dict)

    # getting number of UNIQUE depts in case officer was moved within a dept
    df['number_of_unique_depts'] = df.groupby([id_column])[agency_column].transform("nunique")
    

In [3]:
# getting active officers here
# function that will:
# 1: filter dataframe to have all officers where employment_status is "Active"
# 2: get list of unique ids from this filtered dataframe (this gives us the unique ids of all currently active officers
# 3: create final dataframe that has full employment history of those active officers
# INPUTS
# df: the state's cert data dataframe
# id_column: the name of the column in this dataframe that refers to an officer's unique id
# status_column: the name of the column that refers to officer's status
# active_word: word used in the data to indicate officer is active
# returns: dataframe of the currently active officers' employment history sorted by chronologically by their start dates
def get_active_officers(df, id_column, status_column, active_word):
    active_full_hist = pd.DataFrame()
    active_only = df.loc[df[status_column] == active_word].copy()
    active_ids = active_only[id_column].unique()
    for active in active_ids:
        temp_df = df.loc[df[id_column] == active].copy().sort_values(by='start_date')
        active_full_hist = pd.concat([active_full_hist, temp_df])
    # active_full_hist = df.loc[df[id_column].isin(active_ids)].copy().sort_values([id_column, "start_date"])
    return active_full_hist

In [30]:
# this function returns the same as above function except
# it's in cases where there is no "status" column so we use
# the end date to determine what officers are active. if an officer
# does not have an end date, we assume that they are still active
def get_active_officers_no_status(df, id_column):
    # clean up the end_date column before using it
    df['end_date'] = df['end_date'].fillna('')
    df['start_date'] = df['start_date'].fillna('')

    # now will do similar stuff as with new mexico but using None value rather than Active in status one
    active_full_hist = pd.DataFrame()
    # making dataframe that only has officers who have 'None' in at least one of their end dates
    active_only = df.loc[(df['end_date'] == '') & (df['start_date'] != '')].copy()
    active_ids = active_only[id_column].unique()
    # looping thru list of active ids and getting full employment history of those officers
    for active in active_ids:
        temp_df = df.loc[df[id_column] == active].copy().sort_values(by='start_date')
        active_full_hist = pd.concat([active_full_hist, temp_df])
    return active_full_hist

In [5]:
# this function does the following:
# 1: filters data set to get officers from currently active df that have been terminated
# 2: gets unique ids from that filtered dataframe
# INPUTS
# df: the state's cert data dataframe
# id_column: the name of the column in this dataframe that refers to an officer's unique id
# status_column: the name of the column that refers to officer's status
# term_word: word used in the data to indicate officer was terminated
# returns: dataframe that has full employment history of officers who are currently active
# but have been terminated from a department at some point
def get_terminated_active(active_df, id_column, status_column, term_word):
    terminated_active = active_df.loc[active_df[status_column] == term_word].copy()
    terminated_ids = terminated_active[id_column].unique()
    terminated_active_hist = active_df.loc[active_df[id_column].isin(terminated_ids)].copy()
    return terminated_active_hist

### the two functions below, i use specifically for New Mexico

In [6]:
# looking into what departments come up a lot for terminated active officers
# this function returns dataframe where an officer:
# - was active sometime in the past 5 years
# - had a termination at some point in their career (if there is 1 termination in their history, making sure it's not the last thing)
# the dataframe is sorted by the end date within each officer's employment history
def terminated_active_5yrs(input_df):
    term_active_5yrs = pd.DataFrame()
    final_df = pd.DataFrame()
    input_df['end_date'] = input_df['end_date'].fillna('')
    input_df['end_date'] = pd.to_datetime(input_df['end_date'])
    # dataframe of officers who are either listed as Active or have end date that is 2019 or later
    # we need to filter this down to officers who have ever been Terminated
    active_5_years_back = input_df.loc[(input_df['employment_status'] == 'Active') | (input_df['end_date'].dt.year >= 2019)].copy()
    # unique ids of officers from above dataframe
    ids_active_5_yrs = active_5_years_back['person_nbr'].unique()
    # now we'll look at original new mexico dataframe but only for the ids in the above list and get the ones that have Terminated
    for officer in ids_active_5_yrs:
        temp_df = input_df.loc[(input_df['person_nbr'] == officer) & (input_df['employment_status'] == 'Termination')].copy()
        term_active_5yrs = pd.concat([term_active_5yrs, temp_df])

    # get unique ids of officers who fit the criteria (termination in their employment too)
    ids_active_term = term_active_5yrs['person_nbr'].unique()
    # now we'll look at original new mexico dataframe to get full employment history of officers who have been active at some point
    # in the past 5 years and also have a termination in their career
    for officer in ids_active_term:
        temp_df = (input_df.loc[input_df['person_nbr'] == officer]).sort_values(by='end_date')
        terminations = (temp_df['employment_status'] == 'Termination').sum()
        if (terminations == 1):
            last_row = temp_df.iloc[[-1]]
            if (last_row.iloc[[0]]['employment_status'] != 'Termination').any():
                final_df = pd.concat([final_df, temp_df])
        else:
            final_df = pd.concat([final_df, temp_df])

    return final_df

In [ ]:
# looking into what departments come up a lot for terminated active officers
# this function returns dataframe where an officer:
# - was active sometime in the past 5 years
# - had a termination at some point in their career (if there is 1 termination in their history, making sure it's not the last thing)
# the dataframe is sorted by the end date within each officer's employment history
def terminated_active_5yrs_ill(input_df):
    term_active_5yrs = pd.DataFrame()
    final_df = pd.DataFrame()
    input_df['end_date'] = input_df['end_date'].fillna('')
    input_df['end_date'] = pd.to_datetime(input_df['end_date'], format='mixed', errors='coerce')
    # dataframe of officers who either do not have end date (so active) or have end date that is 2019 or later
    # we need to filter this down to officers who have ever been Terminated
    active_5_years_back = input_df.loc[(input_df['end_date'] == '') | (input_df['end_date'].dt.year >= 2019)].copy()
    # unique ids of officers from above dataframe
    ids_active_5_yrs = active_5_years_back['person_nbr'].unique()
    # now we'll look at original illinois dataframe but only for the ids in the above list and get the ones that have Terminated For Cause
    for officer in ids_active_5_yrs:
        temp_df = input_df.loc[(input_df['person_nbr'] == officer) & (input_df['separation_reason'] == 'Terminated For Cause')].copy()
        term_active_5yrs = pd.concat([term_active_5yrs, temp_df])

    # get unique ids of officers who fit the criteria (termination in their employment too)
    ids_active_term = term_active_5yrs['person_nbr'].unique()
    # now we'll look at original illinois dataframe to get full employment history of officers who have been active at some point
    # in the past 5 years and also have a termination in their career
    for officer in ids_active_term:
        temp_df = (input_df.loc[input_df['person_nbr'] == officer]).sort_values(by='end_date')
        terminations = (temp_df['separation_reason'] == 'Terminated For Cause').sum()
        if (terminations == 1):
            last_row = temp_df.iloc[[-1]]
            if (last_row.iloc[[0]]['separation_reason'] != 'Terminated For Cause').any():
                final_df = pd.concat([final_df, temp_df])
        else:
            final_df = pd.concat([final_df, temp_df])

    return final_df

In [7]:
# this function returns a tuple with two dictionaries
# dictionary 1 has the following:
# key: name of police department
# value: how many times this was the department an officer went to as their next law enforcement job
# after being terminated from a department
# dictionary 2 has the following:
# key: name of police department
# value: list of officers that went there after being terminated from somewhere else
def departments_after_termination(final_df):
    officer_ids = final_df['person_nbr'].unique()
    dept_after_term = defaultdict(int)
    officers_after_term = defaultdict(list)
    final_df['start_date'] = final_df['start_date'].fillna('')
    final_df['start_date'] = pd.to_datetime(final_df['start_date'])

    for officer in officer_ids:
        officer_history = final_df.loc[final_df['person_nbr'] == officer].copy()
        officer_history = officer_history.reset_index(drop=True)
        officer_history = officer_history.sort_values(by='end_date')
        print (officer_history)
        term_spots = officer_history.index[officer_history['employment_status'] == 'Termination']
        print ('TERMINATION INDEX: ', term_spots)

        for term_spot in term_spots:
            # this is seeing if the termination spot is the last one in the officer's history
            # if it is, we are skipping this index because there is no department after in
            # the officer's employment history
            print ('CURRENT TERM SPOT: ', term_spot)
            if (term_spot == officer_history.tail(1).index[0]):
                print ('TERMINATION IS LAST THING IN OFFICERS HISTORY')
            else:
                # making sure officer started at next spot after termination at previous one
                if (officer_history['end_date'].iloc[term_spot] < officer_history['start_date'].iloc[term_spot+1]):
                    department_after_termination = officer_history['agency_name'].iloc[term_spot+1]
                    print ('AGENCY AFTER TERM: ', officer_history['agency_name'].iloc[term_spot+1])
                    dept_after_term[department_after_termination] += 1  
                    officers_after_term[department_after_termination].append(officer)  
                else:
                    print ('OFFICER START DATE AT NEXT PLACE IS BEFORE END DATE AT TERMINATED PLACE') 

    dept_after_term = {k: v for k, v in sorted(dept_after_term.items(), key=lambda item: item[1], reverse=True)}

    return (dept_after_term, officers_after_term)

In [18]:
# this function returns a tuple with two dictionaries
# dictionary 1 has the following:
# key: name of police department
# value: how many times this was the department an officer went to as their next law enforcement job
# after being terminated from a department
# dictionary 2 has the following:
# key: name of police department
# value: list of officers that went there after being terminated from somewhere else
def departments_after_termination_ill(final_df):
    officer_ids = final_df['person_nbr'].unique()
    dept_after_term = defaultdict(int)
    officers_after_term = defaultdict(list)
    final_df['start_date'] = final_df['start_date'].fillna('')
    final_df['start_date'] = pd.to_datetime(final_df['start_date'])

    for officer in officer_ids:
        officer_history = final_df.loc[final_df['person_nbr'] == officer].copy()
        officer_history = officer_history.reset_index(drop=True)
        officer_history = officer_history.sort_values(by='end_date')
        print (officer_history)
        term_spots = officer_history.index[officer_history['separation_reason'] == 'Terminated For Cause']
        print ('TERMINATION INDEX: ', term_spots)

        for term_spot in term_spots:
            # this is seeing if the termination spot is the last one in the officer's history
            # if it is, we are skipping this index because there is no department after in
            # the officer's employment history
            print ('CURRENT TERM SPOT: ', term_spot)
            if (term_spot == officer_history.tail(1).index[0]):
                print ('TERMINATION IS LAST THING IN OFFICERS HISTORY')
            else:
                # making sure officer started at next spot after termination at previous one
                if (officer_history['end_date'].iloc[term_spot] < officer_history['start_date'].iloc[term_spot+1]):
                    department_after_termination = officer_history['agency_name'].iloc[term_spot+1]
                    print ('AGENCY AFTER TERM: ', officer_history['agency_name'].iloc[term_spot+1])
                    dept_after_term[department_after_termination] += 1  
                    officers_after_term[department_after_termination].append(officer)  
                else:
                    print ('OFFICER START DATE AT NEXT PLACE IS BEFORE END DATE AT TERMINATED PLACE') 

    dept_after_term = {k: v for k, v in sorted(dept_after_term.items(), key=lambda item: item[1], reverse=True)}

    return (dept_after_term, officers_after_term)

***

### __New Mexico Analysis__

In [22]:
new_mexico = pd.read_csv('../npi_lookup/national-post-db/nm/input/new-mexico-preprocessed.csv')

In [7]:
new_mexico.sort_values(by='start_date').head(10)

,person_nbr,first_name,middle_name,last_name,suffix,sex,year_of_birth,race,start_date,end_date,employment_type,rank,employment_status,agency_name,agency_type
4516,19-0138-P,Steve,C,Boren,NaN,Male,1980,white,01/23/1208,2022-10-21,Full Time,Cadet,Resigned,Bernalillo Police Department,Law Enforcement Agency
8407,96-0113-P,Joshua,NaN,Valenzuela,NaN,Male,1973,Native American,1911-01-01,1911-01-01,NaN,Patrol Officer,Termination,Jicarilla Apache Tribal Police Department,Law Enforcement Agency
15735,78-0200-P,Glenn,G,Kelsey,Sr.,Male,1951,Native American,1951-03-01,NaN,Full Time,Chief,Active,Pueblo Of Laguna Police Department,Law Enforcement Agency
7305,78-0048-P,Richard,G,Melton,NaN,Male,1957,white,1957-04-14,2004-08-07,NaN,Chief,Resigned,Los Alamos County Police Department,Law Enforcement Agency
10864,81-0077-P,Anselmo,M,Giron,NaN,Male,1921,Hispanic,1957-08-09,1999-06-15,NaN,Chief,Retired,New Mexico State Fair Police,NaN
185,03-0277-P,Ian,P,Conway,NaN,Male,1974,white,1958-09-30,2005-09-30,NaN,Patrol Officer,Resigned,Alamogordo Police Department,Law Enforcement Agency
2905,95-0043-P,Jerry,T,Powell,NaN,Male,1936,white,1962-10-18,1988-06-17,NaN,Deputy Chief,Retired,Albuquerque Police Department,Law Enforcement Agency
1614,03-0269-P,John,R,Dear,NaN,Male,1944,white,1963-06-01,2019-06-21,NaN,Unknown,Resigned,Albuquerque Police Department,Law Enforcement Agency
2501,92-0270-P,Joe,F,Garcia,NaN,Male,1939,Hispanic,1964-09-01,1986-12-31,NaN,Detective,Retired,Albuquerque Police Department,Law Enforcement Agency
13320,64-1007-P,Jose,NaN,Mascarenas,NaN,Male,1939,Hispanic,1964-10-07,1997-06-30,NaN,Sergeant,Retired,New Mexico State Police,Law Enforcement Agency


In [26]:
new_mexico_sorted = new_mexico.sort_values(by=['person_nbr', 'end_date'])

Data goes back to around the 1960s, but important to note the data is messy so it's actually not clear

#### getting active officers

In [25]:
# using previously written function to get # of departments officer has been a part of
get_dept_counts(df=new_mexico, id_column='person_nbr', agency_column='agency_name')

## THIS IS ME TESTING THE FUNCTION TO SEE IF OFFICERS GO TO CERTAIN DEPARTMENTS FREQUENTLY AFTER BEING FIRED

In [27]:
nm_test = terminated_active_5yrs(new_mexico_sorted)

In [28]:
nm_test.head()

,person_nbr,first_name,middle_name,last_name,suffix,sex,year_of_birth,race,start_date,end_date,employment_type,rank,employment_status,agency_name,agency_type,number_of_depts,number_of_unique_depts
14854,00-0022-P,Marvin,L,Paulk,NaN,Male,1967,African American,1998-08-03,1998-08-14,NaN,Deputy Sheriff,Termination,Santa Fe County Sheriff'S Office,Law Enforcement Agency,3,3
14853,00-0022-P,Marvin,L,Paulk,NaN,Male,1967,African American,1999-10-04,2000-06-23,NaN,Patrol Officer,Resigned,Espanola Police Department,Law Enforcement Agency,3,3
14852,00-0022-P,Marvin,L,Paulk,NaN,Male,1967,African American,2000-06-24,2019-01-31,NaN,Captain,Retired,Santa Fe Police Department,Law Enforcement Agency,3,3
17701,00-0034-P,Harry,C,Betz,NaN,Male,1948,white,1999-06-28,2001-06-05,NaN,Agent,Resigned,New Mexico State Police,Law Enforcement Agency,7,7
17702,00-0034-P,Harry,C,Betz,NaN,Male,1948,white,2001-07-23,2006-12-08,NaN,Special Agent,Resigned,New Mexico Attorney General'S Office,Law Enforcement Agency,7,7


In [29]:
nm_depts_after_term = departments_after_termination(nm_test)

  person_nbr first_name middle_name last_name suffix   sex year_of_birth  \
0  00-0022-P     Marvin           L     Paulk    NaN  Male          1967   
1  00-0022-P     Marvin           L     Paulk    NaN  Male          1967   
2  00-0022-P     Marvin           L     Paulk    NaN  Male          1967   

               race start_date   end_date employment_type            rank  \
0  African American 1998-08-03 1998-08-14             NaN  Deputy Sheriff   
1  African American 1999-10-04 2000-06-23             NaN  Patrol Officer   
2  African American 2000-06-24 2019-01-31             NaN         Captain   

  employment_status                       agency_name             agency_type  \
0       Termination  Santa Fe County Sheriff'S Office  Law Enforcement Agency   
1          Resigned        Espanola Police Department  Law Enforcement Agency   
2           Retired        Santa Fe Police Department  Law Enforcement Agency   

   number_of_depts  number_of_unique_depts  
0               

In [31]:
nm_depts_after_term[1]

defaultdict(list,
            {'Espanola Police Department': ['00-0022-P',
              '02-0040-P',
              '04-0050-P',
              '08-0289-P',
              '08-0335-P',
              '11-0092-P',
              '11-0141-P',
              '14-0062-P',
              '18-0358-P',
              '20-0298-P',
              '86-0161-P'],
             'Milan Police Department': ['00-0046-P',
              '03-0044--P',
              '05-0157-P',
              '14-0046-P',
              '14-0186-P'],
             'Gallup Police Department': ['00-0046-P'],
             "Curry County Sheriff'S Office": ['00-0057-P'],
             'Roswell Police Department': ['00-0079-P',
              '03-0160-P',
              '19-0238-P',
              '79-0326-P',
              '86-0124-P'],
             'Portales Police Department': ['00-0079-P', '84-0156-P'],
             'Fort Sumner Police Department': ['00-0080-P', '96-0039-P'],
             'Grants Police Department': ['00-0107-P', '06-0015

In [24]:
rio_arriba_ids = ['00-0286-P',
              '07-0210-P',
              '09-0283-P',
              '12-0296-P',
              '12-0315-P',
              '13-0062-P',
              '14-0006-P',
              '87-0331-P',
              '89-0076-P',
              '90-0053-P',
              '97-0262-P',
              '97-0312-P',
              '98-0187-P']

In [25]:
rio_arriba_county = new_mexico_sorted.loc[new_mexico_sorted['person_nbr'].isin(rio_arriba_ids)].copy()

In [27]:
rio_arriba_county.to_excel('rio_arriba_county_sheriff_officers.xlsx', index=False)

In [32]:
sunland_park_ids = ['00-0198-P',
              '03-0098-P',
              '03-0191-P',
              '06-0121-P',
              '06-0158-P',
              '08-0022-P',
              '09-0044-P',
              '12-0241-P',
              '15-0039-P',
              '87-0279-P',
              '98-0293-P',
              '98-0317-P']

In [33]:
sunland_park = new_mexico_sorted.loc[new_mexico_sorted['person_nbr'].isin(sunland_park_ids)].copy()

In [34]:
sunland_park.to_excel('sunland_park_police_dept_officers.xlsx', index=False)

## END OF THAT TESTING

In [10]:
active_new_mexico = get_active_officers(df=new_mexico, id_column='person_nbr', status_column='employment_status', active_word='Active')

In [11]:
# get active officers who have been at 3+ departments
nm_active_3plus = active_new_mexico.loc[active_new_mexico['number_of_unique_depts'] >= 3].copy()
nm_active_3plus['person_nbr'].nunique()

811

In [12]:
# get active officers who have been at 5+ departments
nm_active_5plus = active_new_mexico.loc[active_new_mexico['number_of_unique_depts'] >= 5].copy()
nm_active_5plus['person_nbr'].nunique()

150

In [13]:
active_new_mexico["person_nbr"].nunique()

5185

There are 5185 unique currently active officers. There are 811 officers who have had 3+ unique employers. There are 150 officers who have been at 5+ departments

In [89]:
# seeing what are the value counts of employment_status
active_new_mexico.value_counts("employment_status")

employment_status
Active           5291
Resigned         2816
Termination       282
Retired           230
Transferred        39
Medical Leave       1
Name: count, dtype: int64

In [14]:
terminated_active_new_mexico = get_terminated_active(active_df=active_new_mexico, id_column='person_nbr', status_column='employment_status', term_word='Termination')
terminated_active_new_mexico["person_nbr"].nunique()

252

Out of the currently active officers, there are 252 officers who have been terminated from a previous department

***

### **North Carolina Analysis**

In [13]:
north_carolina = pd.read_csv('../npi_lookup/national-post-db/nc/input/north-carolina-processed.csv')

#### seeing how far back the data goes

In [27]:
north_carolina.sort_values(by='start_date').head(10)

,start_date,end_date,uid,agency_name,rank,first_name,middle_name,last_name,suffix
88044,1900-01-01,2020-06-30,2831-2787,Buncombe County Sheriff's Office - Deputy Sheriff,Deputy Sheriff,christopher,hall,potter,NaN
8325,1900-01-01,2002-12-31,6799-8742,Hoke County Sheriff's Office - Deputy Sheriff,NaN,lorraine,judson,berry,NaN
97845,1900-01-01,2019-09-20,0056-7322,Siler City Police Department - Law Enforcement...,NaN,john,david,seagroves,NaN
23250,1900-04-28,2017-02-21,0021-7413,Moore County Sheriff's Office - Deputy Sheriff,Deputy Sheriff,john,andrew,conway,NaN
109904,1911-11-11,1911-11-11,4109-3874,Carteret County Sheriff's Office - Deputy Sheriff,NaN,laura,dawn,thompson,NaN
74545,1935-12-28,1979-05-25,6581-0826,Union County Sheriff's Office - Law Enforcemen...,NaN,roy,b,medlin,NaN
115197,1966-11-01,1966-11-01,6350-9772,Johnston County Sheriff's Office - Deputy Sheriff,NaN,worley,aaron,watkins,NaN
548,1970-12-02,1992-08-01,4309-9016,Union County Sheriff's Office - Deputy Sheriff,NaN,hugh,martin,adcock,NaN
46834,1970-12-07,1995-12-18,6999-3201,Buncombe County Sheriff's Office - Deputy Sheriff,NaN,frank,lewis,harris,NaN
78439,1970-12-07,1983-08-31,1010-2855,Buncombe County Sheriff's Office - Law Enforce...,NaN,thomas,harrison,morrissey,NaN


In [28]:
north_carolina.columns

Index(['start_date', 'end_date', 'uid', 'agency_name', 'rank', 'first_name',
       'middle_name', 'last_name', 'suffix'],
      dtype='object')

#### how many currently-active officers have had 3+ emplyers? 5+? More?

In [14]:
get_dept_counts(df=north_carolina, id_column='uid', agency_column='agency_name')

In [30]:
north_carolina.head()

,start_date,end_date,uid,agency_name,rank,first_name,middle_name,last_name,suffix,number_of_depts,number_of_unique_depts
0,1999-12-01,NaN,0012-1559,Asheville Police Department - Law Enforcement ...,Lieutenant,sean,thomas,aardema,NaN,1,1
1,1979-11-02,2005-05-01,0012-1548,Asheville Police Department - Law Enforcement ...,Deputy Sheriff,thomas,frederick,aardema,NaN,3,3
2,2001-04-12,NaN,0012-1548,Buncombe County Sheriff's Office - Deputy Sheriff,Deputy Sheriff,thomas,frederick,aardema,NaN,3,3
3,2011-05-17,2012-06-28,0012-1548,Marion Police Department - Law Enforcement Off...,Deputy Sheriff,thomas,frederick,aardema,NaN,3,3
4,2019-12-19,NaN,2944-5526,Butner Public Safety - Law Enforcement Officer,Public Safety Officer,justin,wayne,aaron,NaN,1,1


we cant get the number of active officers in the same way that we did for new mexico, since there's not status column.
instead, will use when end_date is empty, because that implies officer is still working there.

In [15]:
def get_active_nc():
    # clean up the end_date column before using it
    north_carolina['end_date'] = north_carolina['end_date'].fillna('None')
    north_carolina['start_date'] = north_carolina['start_date'].fillna('None')

    # now will do similar stuff as with new mexico but using None value rather than Active in status one
    active_full_hist = pd.DataFrame()
    # making dataframe that only has officers who have 'None' in at least one of their end dates
    active_only = north_carolina.loc[(north_carolina['end_date'] == 'None') & (north_carolina['start_date'] != 'None')].copy()
    active_ids = active_only['uid'].unique()
    # looping thru list of active ids and getting full employment history of those officers
    for active in active_ids:
        temp_df = north_carolina.loc[north_carolina['uid'] == active].copy().sort_values(by='start_date')
        active_full_hist = pd.concat([active_full_hist, temp_df])
    return active_full_hist

In [16]:
active_north_carolina = get_active_nc()

In [33]:
# get active officers who have been at 3+ departments
nc_active_3plus = active_north_carolina.loc[active_north_carolina['number_of_unique_depts'] >= 3].copy()
nc_active_3plus['uid'].nunique()

6630

In [34]:
# get active officers who have been at 5+ departments
nc_active_5plus = active_north_carolina.loc[active_north_carolina['number_of_unique_depts'] >= 5].copy()
nc_active_5plus['uid'].nunique()

1151

In [17]:
active_north_carolina['uid'].nunique()

34325

There are 34326 unique currently active officers. Out of these, there are 6631 officers who have had 3+ unique employers, and 1151 who have had 5+.

***

### **Minnesota Analysis**

In [18]:
minnesota = pd.read_csv('../npi_lookup/national-post-db/mn/input/minnesota_index.csv')

#### seeing how far back the data goes

In [21]:
minnesota.sort_values(by='start_date').head(20)

,Unnamed: 0,uid,last_name,first_name,middle_name,agency_name,status,start_date,agency_status,end_date
45282,68816,83283,Long,Dennis,Orville,Hendricks Police Department,Terminated,1/1/1900,Former,7/1/1993
46711,71164,84433,Neist,Leslie,Brooke,Adams Police Department,Terminated,1/1/1900,Former,10/1/2011
40222,60593,4380,Franklin,James,De Wayne,West Hennepin Public Safety Department,Terminated,1/1/1976,Former,2/28/1991
39528,59404,3296,Bailey,Larry,Robert,West Hennepin Public Safety Department,Terminated,1/1/1979,Former,12/31/2000
39530,59407,3297,Brotzel,Robert,Joseph,West Hennepin Public Safety Department,Terminated,1/1/1979,Former,2/27/1984
7777,11309,1364,Maslowski,Scott,Alan,Thief River Falls Police Department,Terminated,1/1/1979,Former,4/1/1998
41577,62923,6175,Fossen,Ivan,Owen,Glenwood Police Department,Terminated,1/1/1979,Former,5/27/2000
42063,63698,6616,Mc Kenna,Kevin,Paul,Duluth Police Department,Terminated,1/1/1980,Former,12/31/2002
42062,63696,6615,Johnson,Roger,Keith,Faribault Police Department,Terminated,1/1/1980,Former,5/6/2005
42060,63692,6613,Erspamer,Robert,Thomas,Duluth Police Department,Terminated,1/1/1980,Former,7/30/2005


#### getting active officers

In [19]:
get_dept_counts(df=minnesota, id_column='uid', agency_column='agency_name')

In [24]:
minnesota.shape

(49509, 12)

In [25]:
minnesota.columns

Index(['Unnamed: 0', 'uid', 'last_name', 'first_name', 'middle_name',
       'agency_name', 'status', 'start_date', 'agency_status', 'end_date',
       'number_of_depts', 'number_of_unique_depts'],
      dtype='object')

In [26]:
minnesota.head()

,Unnamed: 0,uid,last_name,first_name,middle_name,agency_name,status,start_date,agency_status,end_date,number_of_depts,number_of_unique_depts
0,1,1,Andersen,Steven,Peder,Verndale Police Department,Terminated,7/15/1973,Former,4/16/1996,4,4
1,2,1,Andersen,Steven,Peder,Beltrami Co. Sheriffs Office,Terminated,5/1/1994,Former,4/27/2007,4,4
2,3,1,Andersen,Steven,Peder,"DPS, Bureau Of Criminal Apprehension",Terminated,7/1/1978,Former,1/1/1980,4,4
3,4,1,Andersen,Steven,Peder,Menahga Police Department,Terminated,4/15/1994,Former,3/1/1995,4,4
4,6,10,Dahl,Gary,Frederick,"DPS, Bureau Of Criminal Apprehension",Terminated,7/1/1978,Former,6/30/1998,1,1


In [20]:
active_minnesota = get_active_officers(df=minnesota, id_column='uid', status_column='status', active_word='Active')

In [31]:
active_minnesota.head()

,Unnamed: 0,uid,last_name,first_name,middle_name,agency_name,status,start_date,agency_status,end_date,number_of_depts,number_of_unique_depts
60,95,10055,Denneson,Richard,James,West Hennepin Public Safety Department,Active,10/17/1988,Primary,NaN,1,1
62,99,10058,Reller,Richard,Herbert,Three Rivers Park District Police Department,Terminated,10/24/1988,Former,12/7/1998,3,3
63,100,10058,Reller,Richard,Herbert,Department Of Natural Resources,Terminated,3/2/1999,Former,3/3/2021,3,3
64,101,10058,Reller,Richard,Herbert,Wright Co. Sheriffs Office,Active,4/7/2021,Primary,NaN,3,3
97,155,10080,Cleveland,Kent,Alan,Hennepin Co. Sheriffs Office,Terminated,10/31/1988,Former,10/27/1989,6,6


In [32]:
# get active officers who have been at 3+ departments
mn_active_3plus = active_minnesota.loc[active_minnesota['number_of_unique_depts'] >= 3].copy()
mn_active_3plus['uid'].nunique()

2154

In [33]:
# get active officers who have been at 5+ departments
mn_active_5plus = active_minnesota.loc[active_minnesota['number_of_unique_depts'] >= 5].copy()
mn_active_5plus['uid'].nunique()

415

In [34]:
active_minnesota['uid'].nunique()

10683

***

### **Indiana Analysis** 

In [2]:
indiana = pd.read_csv('../npi_lookup/national-post-db/in/output/indiana_index.csv')

In [4]:
indiana.shape

(68247, 11)

In [7]:
indiana.sort_values(by='start_date').head(250)

,person_nbr,agency_name,last_name,first_name,gender,race,birth_year,start_date,end_date,rank,separation_reason
533,3079-1089,Randolph County Sheriff's Department,Alexander,Rick,Male,NaN,1948.0,1900-01-01,1900-01-01,Jail Officer,Other
55,9261-6819,Grant County Sheriff's Department,Abernathy,Mary,Female,NaN,1948.0,1900-01-01,2013-08-01,Jail Officer,Deceased
80,4612-5103,Rush County Sheriff's Department,Abraham,J.,NaN,NaN,NaN,1900-01-01,1998-02-15,Jail Officer,Separated
17315,2964-5645,Marion County Sheriff's Department,Graffitti,Diane,Female,NaN,1976.0,1900-01-01,2003-02-15,Reserve Deputy,Separated
37104,2706-2578,Indiana Department of Corrections,Purviance,Lester,Male,NaN,1955.0,1900-01-01,2015-12-03,Unknown,Separated
44340,4759-7476,White County Sheriff's Department,Stevenson,Luke,Male,NaN,1996.0,1900-01-01,2023-08-28,Jail Officer,Resigned
43235,6015-9855,Warren County Sheriff's Office,Snedeker,Paula,Female,White,1971.0,1900-01-01,2014-10-09,Jail Officer,Discharged
51390,6048-9612,Columbia City Police Department,Wright,Nora,Female,White,1987.0,1921-01-02,2022-04-09,Dispatcher,Resigned
31706,7319-1016,Greene County Sheriff's Department,Milligan,Logan,Female,White,1997.0,1921-05-10,2023-03-28,Officer,Resigned
33193,2605-2036,DeKalb County Sheriff's Department,Myers,Arien,Female,NaN,1978.0,1923-02-14,NaN,Jail Officer,Active


data isnt great for determining exactly how far back it goes

#### getting active officers

In [8]:
get_dept_counts(df=indiana, id_column='person_nbr', agency_column='agency_name')

In [9]:
indiana.shape

(68247, 13)

In [10]:
indiana.columns

Index(['person_nbr', 'agency_name', 'last_name', 'first_name', 'gender',
       'race', 'birth_year', 'start_date', 'end_date', 'rank',
       'separation_reason', 'number_of_depts', 'number_of_unique_depts'],
      dtype='object')

In [11]:
active_indiana = get_active_officers(df=indiana, id_column='person_nbr', status_column='separation_reason', active_word='Active')

In [12]:
active_indiana.to_excel('indiana/active_indiana.xlsx', index=False)

In [13]:
active_indiana['person_nbr'].nunique()

20507

there's 20507 currently active officers in indiana

In [13]:
# get active officers who have been at 3+ departments
in_active_3plus = active_indiana.loc[active_indiana['number_of_unique_depts'] >= 3].copy()
in_active_3plus['person_nbr'].nunique()

2074

In [14]:
# get active officers who have been at 5+ departments
in_active_5plus = active_indiana.loc[active_indiana['number_of_unique_depts'] >= 5].copy()
in_active_5plus['person_nbr'].nunique()

206

In [15]:
in_active_3plus.to_excel('indiana/in_active_3plus.xlsx', index=False)
in_active_5plus.to_excel('indiana/in_active_5plus.xlsx', index=False)

There are 20507 unique currently active officers. Out of these, there are 2074 officers who have had 3+ unique employers, and 206 who have had 5+.

In [16]:
active_indiana.value_counts('separation_reason')

separation_reason
Active                     21445
Resigned                    6732
Separated                   1436
Retired                      804
Other                        546
Discharged                   290
Never with Organization       22
Deceased                      10
Revoked                        8
On Leave                       3
Name: count, dtype: int64

In [31]:
terminated_active_indiana = get_terminated_active(active_df=active_indiana, id_column='person_nbr', status_column='separation_reason', term_word='Discharged')
terminated_active_indiana['person_nbr'].nunique()

271

Out of the currently active officers, there are 271 officers who have been terminated from a previous department

### **Kansas Analysis**

In [32]:
kansas = pd.read_csv('../npi_lookup/national-post-db/ks/output/ks-2024-index.csv')

In [99]:
kansas.head(5)

,person_nbr,full_name,last_name,first_name,middle_name,middle_initial,suffix,agency_name,rank,status,start_date,end_date
0,18468,Jeffery Ridgway,Ridgway,Jeffery,NaN,NaN,NaN,Ellis County Sheriff's Office,Detective,F,12/31/2023,NaN
1,34726,Clayton Philpott,Philpott,Clayton,NaN,NaN,NaN,Marion County Sheriff's Office,Deputy,F,12/29/2023,NaN
2,33426,Juliet Gray,Gray,Juliet,NaN,NaN,NaN,Linn County Sheriff's Office,Deputy,F,12/29/2023,NaN
3,21555,Joe Sutton,Sutton,Joe,NaN,NaN,NaN,Cherokee Police Department,Patrol Officer,F,12/28/2023,NaN
4,31520,Justin Wunder,Wunder,Justin,NaN,NaN,NaN,Auburn-Washburn USD #437 School Police,Patrol Officer,F,12/27/2023,NaN


In [100]:
kansas.sort_values(by='start_date').head(40)

,person_nbr,full_name,last_name,first_name,middle_name,middle_initial,suffix,agency_name,rank,status,start_date,end_date
21899,13427,Bradley Ricke,Ricke,Bradley,NaN,NaN,NaN,Ellis County Sheriff's Office,Deputy,F,1/1/1996,NaN
21714,15863,Jason Sutton,Sutton,Jason,NaN,NaN,NaN,Kansas City Kansas Police Department,Detective,F,1/1/1999,NaN
21472,13605,William Robinson,Robinson,William,NaN,NaN,NaN,Clay Center Police Department,Chief,F,1/1/2002,NaN
21293,21262,Sheri Staab,Staab,Sheri,NaN,NaN,NaN,Thomas County Sheriff's Office,Deputy,P,1/1/2004,NaN
21294,11034,Travis Mishler,Mishler,Travis,NaN,NaN,NaN,Lyon County Sheriff's Office,Detective,F,1/1/2004,NaN
21163,15803,Shelby Sturgeon,Sturgeon,Shelby,NaN,NaN,NaN,Topeka Police Department,Lieutenant,F,1/1/2005,NaN
21055,14151,Christopher Scheuber,Scheuber,Christopher,NaN,NaN,NaN,Augusta Department of Public Safety,Sergeant,F,1/1/2006,NaN
21054,19523,Darrin Littlepage,Littlepage,Darrin,NaN,NaN,NaN,Kansas Highway Patrol,trpr,F,1/1/2006,NaN
20878,3236,William Cory,Cory,William,NaN,NaN,NaN,Lawrence Police Department,Captain,F,1/1/2007,NaN
20877,23065,Todd Walsh,Walsh,Todd,NaN,NaN,NaN,Saline County Sheriff's Office,Deputy,F,1/1/2007,NaN


In [101]:
kansas.columns

Index(['person_nbr', 'full_name', 'last_name', 'first_name', 'middle_name',
       'middle_initial', 'suffix', 'agency_name', 'rank', 'status',
       'start_date', 'end_date'],
      dtype='object')

In [33]:
get_dept_counts(df=kansas, id_column='person_nbr', agency_column='agency_name')

we cant get the number of active officers in the same way that we did for new mexico, since there's not status column.
instead, will use when end_date is empty, because that implies officer is still working there.

In [34]:
active_kansas = get_active_officers_no_status(df=kansas, id_column='person_nbr')

In [105]:
active_kansas['person_nbr'].nunique()

9812

In [106]:
# get active officers who have been at 3+ departments
ks_active_3plus = active_kansas.loc[active_kansas['number_of_unique_depts'] >= 3].copy()
ks_active_3plus['person_nbr'].nunique()

836

In [107]:
# get active officers who have been at 3+ departments
ks_active_5plus = active_kansas.loc[active_kansas['number_of_unique_depts'] >= 5].copy()
ks_active_5plus['person_nbr'].nunique()

112

### **Mississippi Analysis**

In [35]:
mississippi = pd.read_csv('../npi_lookup/national-post-db/ms/output/mississippi-processed.csv')

In [36]:
mississippi.head()

,first_name,last_name,Agency,TYPE,employment_status,start_date,end_date,person_nbr,agency_name
0,Timothy,Pruitt,Abbeville,Chief,Part-Time,2010-04-01,NaN,13792,abbeville chief
1,Jimmy,Easley,Abbeville,Chief,Part-Time,1995-05-15,2009-11-30,14876,abbeville chief
2,Jacob,Abel,ABC,State,Full-Time,2022-05-02,NaN,19785,abc state
3,Kurt,Alexander,ABC,State,Full-Time,2010-09-13,NaN,21028,abc state
4,William,Alexander,ABC,State,Full-Time,2002-07-01,2019-09-03,15761,abc state


In [96]:
mississippi.sort_values(by='start_date').head(20)

,first_name,last_name,Agency,TYPE,employment_status,start_date,end_date,person_nbr,agency_name
28708,Olaf,Raybon,Picayune,PD,Part-Time,1962-10-15,NaN,13218,picayune police department
36984,William,Lloyd,Wildlife,State,Full-Time,1966-09-23,2007-06-30,2973,wildlife state
763,Michael,Kingdom,Arcola,PD,Part-Time,1970-01-18,2008-06-30,19589,arcola police department
592,Milton,Bishop,Alcorn Co,SO,Part-Time,1970-01-20,2012-02-01,13277,alcorn county sheriff's office
16849,Hilman,Wedgeworth,Jones Co,SO,Part-Time,1970-06-20,2008-01-07,13211,jones county sheriff's office
36623,Hubert,Nabors,West Point,PD,Part-Time,1970-11-07,2008-12-05,5170,west point police department
26046,Robert,Germany,Ocean Springs,PD,Full-Time,1970-12-01,2005-01-27,4285,ocean springs police department
22783,Robert,Tinsley,Meridian,PD,Full-Time,1971-05-24,2013-10-01,1184,meridian police department
27430,Thomas,Ruspoli,Pass Christian,PD,Full-Time,1971-12-07,2011-12-08,1702,pass christian police department
24812,Abe,Banks,Moss Point,PD,Part-Time,1972-03-29,2010-11-10,14980,moss point police department


In [97]:
mississippi.columns

Index(['first_name', 'last_name', 'Agency', 'TYPE', 'employment_status',
       'start_date', 'end_date', 'person_nbr', 'agency_name'],
      dtype='object')

In [37]:
get_dept_counts(df=mississippi, id_column='person_nbr', agency_column='agency_name')

In [38]:
active_mississippi = get_active_officers_no_status(df=mississippi, id_column='person_nbr')

In [110]:
active_mississippi['person_nbr'].nunique()

9657

In [111]:
# get active officers who have been at 3+ departments
ms_active_3plus = active_mississippi.loc[active_mississippi['number_of_unique_depts'] >= 3].copy()
ms_active_3plus['person_nbr'].nunique()

2413

In [112]:
# get active officers who have been at 3+ departments
ms_active_5plus = active_mississippi.loc[active_mississippi['number_of_unique_depts'] >= 5].copy()
ms_active_5plus['person_nbr'].nunique()

421

***

### __Illinois Analysis__

In [35]:
illinois = pd.read_csv('../npi_lookup/national-post-db/il/input/illinois_index.csv')

In [36]:
illinois_sorted = illinois.sort_values(by=['person_nbr', 'end_date'])

In [39]:
illinois_sorted.head()

,person_nbr,last_name,first_name,middle_name,race,sex,year_of_birth,agency_name,start_date,end_date,separation_reason,type,status,rank,number_of_depts,number_of_unique_depts
174985,65000001,A'Hearn,Daniel,John,White,Male,1944,Will County Sheriff's Office,1986-01-07,1986-10-06,Resigned,Law Enforcement,Full-Time,Deputy,1,1
174986,65000002,A'Hearn,Jo,Ann,White,Female,1946,Knox County Sheriff's Office,1979-04-09,2010-01-01,Resigned,Law Enforcement,Full-Time,Deputy,1,1
174987,65000003,Aalto,Brian,John,White,Male,1978,McHenry Police Department,2001-01-05,NaN,NaN,Law Enforcement,Full-Time,Sergeant,1,1
174988,65000004,Aanonsen,Linda,Kay,White,Female,1954,DuPage County Sheriff's Office,1998-12-28,2011-09-26,Other (Explain),Correctional,Full-Time,Deputy,2,1
174989,65000004,Aanonsen,Linda,Kay,White,Female,1954,DuPage County Sheriff's Office,2011-09-26,2019-06-03,Retired,Court Security,Full-Time,Court Security Officer,2,1


#### getting active officers

In [38]:
# using previously written function to get # of departments officer has been a part of
get_dept_counts(df=illinois_sorted, id_column='person_nbr', agency_column='agency_name')

In [12]:
active_illinois = get_active_officers_no_status(df=illinois, id_column='person_nbr')

In [13]:
active_illinois['person_nbr'].nunique()

44892

there's 44,892 currently active officers in Illinois

In [13]:
# get active officers who have been at 3+ departments
il_active_3plus = active_illinois.loc[active_illinois['number_of_unique_depts'] >= 3].copy()
il_active_3plus['person_nbr'].nunique()

6106

In [14]:
# get active officers who have been at 3+ departments
il_active_5plus = active_illinois.loc[active_illinois['number_of_unique_depts'] >= 5].copy()
il_active_5plus['person_nbr'].nunique()

1466

Out of the 44,892 currently active officers in Illinois, 6,106 of them have had 3+ unique employers, and 1,466 have had 5+ unique employers.

In [15]:
active_illinois.value_counts('separation_reason')

separation_reason
Resigned                             19595
Other (Explain)                       8532
Unknown                               2606
Left For New Job                      1961
Retired                               1956
Terminated For Cause                  1169
No Separation On File At Board         464
Removed From Roster By Department      175
Agency Disbanded                       156
Automated                               71
Deceased                                11
Charged/Convicted Of Crime               9
Disabled                                 1
Name: count, dtype: int64

In [16]:
terminated_active_illinois = get_terminated_active(active_df=active_illinois, id_column='person_nbr', status_column='separation_reason', term_word='Terminated For Cause')
terminated_active_illinois['person_nbr'].nunique()

1005

In [14]:
illinois.sort_values(by='start_date').head(10)

,person_nbr,last_name,first_name,middle_name,race,sex,year_of_birth,agency_name,start_date,end_date,separation_reason,type,status,rank
225912,65162444,Clover,Bryce,A,White,Male,2001,Jackson County Sheriff's Office,0203-07-24,2024-06-18,Resigned,Correctional,Part-Time,Correctional Officer
6739,65007833,Blake,Chad,Anthony,White,Male,1972,PTB Intern Program,1900-01-01,2010-01-01,Other (Explain),Other,Unknown,Intern
108493,65003915,Baker,Rolf,Fedder,White,Male,1974,PTB Intern Program,1900-01-01,2010-01-01,Other (Explain),Other,Unknown,Intern
223773,65105589,Williams,Eloise,NaN,Native American or Alaska Native,Male,1901,Chicago Housing Authority Police Department,1900-01-01,1999-09-01,Agency Disbanded,Law Enforcement,Full-Time,Police Officer
121698,65066962,Moore,Michael,Shane,White,Male,1975,PTB Intern Program,1900-01-01,2010-01-01,Other (Explain),Other,Unknown,Intern
182850,65049657,Kando,Danny,E,White,Male,1900,PTB Intern Program,1900-01-01,2010-01-01,Other (Explain),Other,Unknown,Intern
167482,65030127,Fletcher,Brian,Joseph,White,Male,1977,PTB Intern Program,1900-01-01,2010-01-01,Other (Explain),Other,Unknown,Intern
9052,65025334,Draper,Jason,E,White,Male,1975,PTB Intern Program,1900-01-01,2010-01-01,Other (Explain),Other,Unknown,Intern
171632,65105959,Williams,Thomas,Lee,White,Male,1977,PTB Intern Program,1900-01-01,2010-01-01,Other (Explain),Other,Unknown,Intern
6308,65003487,Backshis,John,N,White,Male,1957,Maywood Police Department,1900-01-01,2019-01-01,No Separation On File At Board,Other,Unknown,Unspecified


In [15]:
illinois_sorted.head()

,person_nbr,last_name,first_name,middle_name,race,sex,year_of_birth,agency_name,start_date,end_date,separation_reason,type,status,rank,number_of_depts,number_of_unique_depts
174985,65000001,A'Hearn,Daniel,John,White,Male,1944,Will County Sheriff's Office,1986-01-07,1986-10-06,Resigned,Law Enforcement,Full-Time,Deputy,1,1
174986,65000002,A'Hearn,Jo,Ann,White,Female,1946,Knox County Sheriff's Office,1979-04-09,2010-01-01,Resigned,Law Enforcement,Full-Time,Deputy,1,1
174987,65000003,Aalto,Brian,John,White,Male,1978,McHenry Police Department,2001-01-05,NaN,NaN,Law Enforcement,Full-Time,Sergeant,1,1
174988,65000004,Aanonsen,Linda,Kay,White,Female,1954,DuPage County Sheriff's Office,1998-12-28,2011-09-26,Other (Explain),Correctional,Full-Time,Deputy,2,1
174989,65000004,Aanonsen,Linda,Kay,White,Female,1954,DuPage County Sheriff's Office,2011-09-26,2019-06-03,Retired,Court Security,Full-Time,Court Security Officer,2,1


In [16]:
illinois_sorted.value_counts('separation_reason')

separation_reason
Resigned                             60026
Unknown                              44590
Retired                              27362
Other (Explain)                      19373
Terminated For Cause                  7684
No Separation On File At Board        7105
Left For New Job                      6285
Automated                             2788
Agency Disbanded                      2518
Deceased                              1546
Removed From Roster By Department     1306
Charged/Convicted Of Crime             181
Disabled                                41
Disqualified Part Time L. E.             4
Decertified By Board                     1
Name: count, dtype: int64

In [46]:
ill_test = terminated_active_5yrs_ill(illinois_sorted)

In [55]:
ill_test.head(20)

,person_nbr,last_name,first_name,middle_name,race,sex,year_of_birth,agency_name,start_date,end_date,separation_reason,type,status,rank,number_of_depts,number_of_unique_depts
54417,65000149,Abukhudair,Gamal,NaN,White,Male,1982,University Of Chicago Police Department,2011-12-30,2012-04-24,Terminated For Cause,Law Enforcement,Full-Time,Police Officer,2,2
54418,65000149,Abukhudair,Gamal,NaN,White,Male,1982,University of Illinois at Chicago Campus Polic...,2022-01-06,2022-12-13,Resigned,Law Enforcement,Full-Time,Police Officer,2,2
161890,65000264,Adam,Clifford,L,Black or African American,Male,1959,Abingdon Police Department,1998-12-07,2001-12-14,Left For New Job,Law Enforcement,Part-Time,Police Officer,14,8
161888,65000264,Adam,Clifford,L,Black or African American,Male,1959,Warren County Sheriff's Office,1992-06-29,2010-01-01,Unknown,Law Enforcement,Full-Time,Sergeant,14,8
161891,65000264,Adam,Clifford,L,Black or African American,Male,1959,Knoxville Police Department,2001-11-19,2013-05-17,Other (Explain),Law Enforcement,Part-Time,Police Officer,14,8
161889,65000264,Adam,Clifford,L,Black or African American,Male,1959,Alexis Police Department,1997-03-25,2015-02-10,Terminated For Cause,Law Enforcement,Part-Time,Police Officer,14,8
161896,65000264,Adam,Clifford,L,Black or African American,Male,1959,Warren County Sheriff's Office,2008-03-19,2016-10-13,Resigned,Law Enforcement,Part-Time,Deputy,14,8
161898,65000264,Adam,Clifford,L,Black or African American,Male,1959,Oquawka Police Department,2016-10-23,2017-10-10,Other (Explain),Law Enforcement,Part-Time,Police Officer,14,8
161899,65000264,Adam,Clifford,L,Black or African American,Male,1959,Oquawka Police Department,2017-12-11,2021-07-01,Other (Explain),Law Enforcement,Full-Time,Chief,14,8
161892,65000264,Adam,Clifford,L,Black or African American,Male,1959,Henderson County Sheriff's Office,2022-08-05,2022-09-06,Other (Explain),Law Enforcement,Part-Time,Deputy,14,8


In [51]:
ill_depts_after_term = departments_after_termination_ill(ill_test)

   person_nbr   last_name first_name middle_name   race   sex  year_of_birth  \
0    65000149  Abukhudair      Gamal         NaN  White  Male           1982   
1    65000149  Abukhudair      Gamal         NaN  White  Male           1982   

                                         agency_name start_date   end_date  \
0            University Of Chicago Police Department 2011-12-30 2012-04-24   
1  University of Illinois at Chicago Campus Polic... 2022-01-06 2022-12-13   

      separation_reason             type     status            rank  \
0  Terminated For Cause  Law Enforcement  Full-Time  Police Officer   
1              Resigned  Law Enforcement  Full-Time  Police Officer   

   number_of_depts  number_of_unique_depts  
0                2                       2  
1                2                       2  
TERMINATION INDEX:  Index([0], dtype='int64')
CURRENT TERM SPOT:  0
AGENCY AFTER TERM:  University of Illinois at Chicago Campus Police Department
    person_nbr  last_name fi

In [74]:
ill_depts_after_term[1]

defaultdict(list,
            {'University of Illinois at Chicago Campus Police Department': [65000149],
             "Washington County Sheriff's Office": [65000690, 65110655],
             'Oakwood Police Department': [65000889, 65022657],
             'Berkeley Police Department': [65000893,
              65003334,
              65021824,
              65151295],
             'Mounds Police Department': [65000909,
              65033991,
              65072714,
              65098800,
              65104535,
              65119816,
              65120112],
             'Bloomington Police Department': [65001415],
             'Toluca Police Department': [65001415, 65006749, 65118640],
             "Lake County Sheriff's Office": [65001594,
              65002025,
              65009144,
              65011218,
              65040396,
              65063382],
             'Markham Police Department': [65001818,
              65011135,
              65033406,
              65044544,
 

In [64]:
dixmoo_ids = [65002681,
              65005628,
              65012197,
              65012197,
              65012198,
              65012645,
              65033927,
              65036239,
              65038177,
              65040126,
              65045290,
              65046359,
              65046644,
              65064009,
              65066795,
              65082037,
              65084128,
              65084128,
              65084558,
              65087282,
              65090921,
              65098037,
              65098384,
              65112930,
              65114578,
              65125986,
              65144416,
              65144416,
              65148595,
              65152011]

In [65]:
dixmoor_officers = illinois_sorted.loc[illinois_sorted['person_nbr'].isin(dixmoo_ids)].copy()

In [67]:
dixmoor_officers.to_excel('illinois/dixmoor_officers.xlsx', index=False)

In [69]:
riverdale_ids = [65044192, 65107502, 65138146]

In [70]:
riverdale_officers = illinois_sorted.loc[illinois_sorted['person_nbr'].isin(riverdale_ids)].copy()

In [72]:
riverdale_officers.to_excel('illinois/riverdale_officers.xlsx')

In [75]:
dolton_ids = [65061174,
              65069637,
              65080137,
              65087282,
              65088471,
              65097133,
              65138146]

dolton_officers = illinois_sorted.loc[illinois_sorted['person_nbr'].isin(dolton_ids)].copy()